In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop ,SGD


from eXplainableAI.transparent_model.RNN import ConditionalRETAIN
from eXplainableAI.transparent_model.RNN.interpretation import Interpreter
from eXplainableAI.transparent_model.RNN.evaluation import model_week_evaluation, mean_absolute_percentage_error

In [ ]:
week_labels = ['{}week'.format(i) for i in range(1, 17)]

In [ ]:
# train
X_train_var = np.load('./cache/numpy/X_train_var.npy')
X_train_inv = np.load('./cache/numpy/X_train_fix.npy')
Y_train = np.load('./cache/numpy/Y_train.npy')

# test
X_test_var = np.load('./cache/numpy/X_test_var.npy')
X_test_inv = np.load('./cache/numpy/X_test_fix.npy')
Y_test = np.load('./cache/numpy/Y_test.npy')

# Remove weight diff
X_train_var = np.delete(X_train_var, -6, axis=2)
X_test_var = np.delete(X_test_var, -6, axis=2)

vars_ = X_train_var.shape[-1]

print(X_train_var.shape, X_train_inv.shape)

vars_ = ['wt_frequency','meal_adherence','calories','steps',
         'n_exercise',
         'n_drink','n_overcal',
        'Other', 'SharplyDecrease', 'Yoyo', 'Increase', 'ModerateDecrease']
inv_ = ['Female', 'Male', 'age', 'height', 'initKg',
       'Obese class I', 'Obese class II', 'Obese class III', 'Overweight']

In [ ]:
config = {'n_features':len(vars_),
          'n_aux':len(inv_),
          'step':16,
          'hidden_units':20}

retain = ConditionalRETAIN(config)
model = retain.build_model()
model.compile(optimizer=RMSprop(lr=0.05), loss='mse', metrics=['mse'])
model.summary()

In [ ]:
y_predict = model.predict(x=[X_test_var, X_test_inv])

MAPE = mean_absolute_percentage_error(Y_test, y_predict)
print('MAPE (Overall): {} %'.format(MAPE))

performance = model_week_evaluation(model, x=[X_test_var, X_test_inv], y=Y_test)
performance = [round(i, 5) for i in performance]
print(performance)

### Alpha

In [ ]:
interpreter = Interpreter(model, X_test_var, X_test_inv, Y_test)
alpha = np.array(interpreter.get_model_weight('alpha'))

alpha_mean = alpha.reshape(-1, 16, 16, 1)[:, -1, :, :].mean(axis=0)
print('Alpha shape: ', alpha_mean.shape)

fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.heatmap(alpha_mean.T, annot=True, fmt='.4f', xticklabels=list(range(1, 17)))

### Beta

In [ ]:
beta = np.array(interpreter.get_model_weight('beta'))
beta = beta.reshape(-1, 16, 16, len(vars_))
beta_mean = beta[:, 15, :, :].mean(axis=0)

fig, axes = plt.subplots(1, 1, figsize=(13, 6))
sns.heatmap(beta_mean.T, 
            cmap='coolwarm', 
            annot=True, 
            vmin=-1, 
            vmax=1, 
            xticklabels=list(range(1, 17)),
            yticklabels=vars_
           )
plt.xticks(rotation=45)

### Dense layer

In [ ]:
W, bias = interpreter.get_model_weight('weight')
W.reshape(-1, W.shape[0])[0][2:4] = W.reshape(-1, W.shape[0])[0][2:4] * 1000  # # calories, steps * 1K당으로 변경


fig, axes = plt.subplots(1, 1, figsize=(18, 6))
sns.heatmap(W.reshape(-1, W.shape[0]), cmap='coolwarm', fmt='.3f', annot=True, 
            xticklabels=vars_+inv_,
            vmin=-1, vmax=1)

fig.savefig('./results/denselayer.png')

### Overall Coef contribution

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(14, 7))

W_time = W[:len(vars_)]
contr_coef = W_time.ravel() * (alpha_mean * beta_mean)

sns.heatmap(contr_coef.T, cmap='coolwarm', annot=True, fmt='.3f',
            yticklabels=vars_,
            xticklabels=list(range(1, 17)),
            vmin=-0.06, 
            vmax=0.06)